# Advanced Topics in Machine Learning - Natural Language Processing -- Group Assignment
> Students
> - Patrik Rác
> - Méline Trochon
> - Valentina Moretti
> - Mattia Colbertaldo

---



In [47]:
# Imports
import os
import torch
import pandas

## 1. Investigate Dataset


We will investigate the dataset ***SQuAD2.0 (Stanford Question Answering Dataset)*** avaliable [here](https://rajpurkar.github.io/SQuAD-explorer/).

In [48]:
# Set the directory to work in
WORKING_DIR = "./squad"
DATA_DIR = "./data"

In [23]:
# Download the dataset
print("Downloading the DEV dataset of SQuAD2.0")
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2024-01-03 11:19:55--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       100%[===================>]   4.17M  --.-KB/s    in 0.05s   

2024-01-03 11:19:56 (85.3 MB/s) - ‘dev-v2.0.json’ saved [4370528/4370528]



In [ ]:
dev_dataframe = pandas.read_json("/content/dev-v2.0.json")
print(f"Size of the dataset: {dev_dataframe.size} (e.g. Categories of questions)")
dev_dataframe.head()

Size of the dataset: 70 (e.g. Categories of questions)


,version,data
0,v2.0,"{'title': 'Normans', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Computational_complexity_theory', '..."
2,v2.0,"{'title': 'Southern_California', 'paragraphs':..."
3,v2.0,"{'title': 'Sky_(United_Kingdom)', 'paragraphs'..."
4,v2.0,"{'title': 'Victoria_(Australia)', 'paragraphs'..."


In [ ]:
def count_paragraphs(row):
    return len(row["data"]["paragraphs"])

print(f"Number of individual paragraphs: {sum(dev_dataframe.apply(count_paragraphs, axis=1))}")


def count_questions(row):
  return sum([len(p["qas"]) for p in row["data"]["paragraphs"]])

print(f"Number of individual questions: {sum(dev_dataframe.apply(count_questions, axis=1))}")

def count_impossible_questions(row):
  n_impossible = 0
  for p in row["data"]["paragraphs"]:
    for q in p["qas"]:
      if q["is_impossible"]:
        n_impossible += 1
  return n_impossible


print(f"Number of impossible questions: {sum(dev_dataframe.apply(count_impossible_questions, axis=1))}")

Number of individual paragraphs: 1204
Number of individual questions: 11873
Number of impossible questions: 5945


In [ ]:
print(type(dev_dataframe.iloc[0]["data"]))

import json
print(json.dumps(dev_dataframe.iloc[1]["data"], sort_keys=False, indent=4))

<class 'dict'>
{
    "title": "Computational_complexity_theory",
    "paragraphs": [
        {
            "qas": [
                {
                    "question": "What branch of theoretical computer science deals with broadly classifying computational problems by difficulty and class of relationship?",
                    "id": "56e16182e3433e1400422e28",
                    "answers": [
                        {
                            "text": "Computational complexity theory",
                            "answer_start": 0
                        },
                        {
                            "text": "Computational complexity theory",
                            "answer_start": 0
                        },
                        {
                            "text": "Computational complexity theory",
                            "answer_start": 0
                        }
                    ],
                    "is_impossible": false
                },
           

In [ ]:
# Example of a Paragraph and a question
import textwrap
print("Category of given paragraph: {}\n".format(dev_dataframe.iloc[0]["data"]["title"]))

example_datum = dev_dataframe.iloc[0]["data"]["paragraphs"][0]
print("Context\n {}\n".format(textwrap.fill(example_datum["context"], 50)))

print("Example Question:\n{}\n".format(example_datum["qas"][0]["question"]))

print("Example Answer:\n{}\n".format(example_datum["qas"][0]["answers"][0]["text"]))

Category of given paragraph: Normans

Context
 The Normans (Norman: Nourmands; French: Normands;
Latin: Normanni) were the people who in the 10th
and 11th centuries gave their name to Normandy, a
region in France. They were descended from Norse
("Norman" comes from "Norseman") raiders and
pirates from Denmark, Iceland and Norway who,
under their leader Rollo, agreed to swear fealty
to King Charles III of West Francia. Through
generations of assimilation and mixing with the
native Frankish and Roman-Gaulish populations,
their descendants would gradually merge with the
Carolingian-based cultures of West Francia. The
distinct cultural and ethnic identity of the
Normans emerged initially in the first half of the
10th century, and it continued to evolve over the
succeeding centuries.

Example Question:
In what country is Normandy located?

Example Answer:
France



## Finetuning DistilBERT

In [ ]:
# Install the required Huggingface libs
! pip install datasets transformers accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [49]:
# Import the transformers package (We'll be taking a pretrained model from here)
import transformers
print(transformers.__version__)

4.22.2


In [ ]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [ ]:
from datasets import load_dataset

squad2_datasets = load_dataset("squad_v2") 

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [50]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

NameError: ignored

In [51]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

AssertionError: ignored

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

NameError: ignored

### Actual Fine Tuning of the Model

In [52]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

RuntimeError: ignored

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

NameError: ignored

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("test-squad-trained")

In [ ]:
# TODO: Evaluation. The evaluation of the model should be rather genereal and work with all model training strategies.

# One shot learning

### QA data preparation

In this section we will be using the [WikiQA](https://aclanthology.org/D15-1237/) data set.
It's a data set for open domain generative QA.

It's avaialble via the HuggingFace data set package, let's install it

In [3]:
!pip -q install datasets

In [2]:
!pip -q install transformers==4.22.2
!pip -q install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.9 MB/s eta 0:00:00


In [4]:
!pip install --upgrade accelerate

ERROR: Operation cancelled by user


Let's download the validation split of the dev_dataframe data set

In [5]:
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
import datasets

The data set contains questions, the answers and the paragraphs.
Some questions are impossible and don't have answers.

This cell must be run only if one wants to filter the data to retain only correct question-response pairs.
However we consider impossible answers good to have since we want to test how the model behaves on them.

In [7]:
import pandas as pd

dev_dataframe = pd.read_json("/content/dev-v2.0.json")

# Flatten the nested structure to work with individual questions
flat_dataframe = pd.json_normalize(dev_dataframe['data'], record_path=['paragraphs', 'qas'], meta=['title', 'paragraphs'])



def remove_impossible_questions(row, possible_questions_list):
    count = 0
    for p in row["data"]["paragraphs"]:
        for q in p["qas"]:
            if not q["is_impossible"]:
                possible_questions_list.append(q)
                count += 1
    # print("Number of possible questions:", count)


dev_dataframe_possible_list = []
dev_dataframe.apply(remove_impossible_questions, axis=1, possible_questions_list=dev_dataframe_possible_list)
print(len(dev_dataframe_possible_list))
# Convert the list of dictionaries to a DataFrame
dev_dataframe_possible = pd.DataFrame(dev_dataframe_possible_list)

# Display the first few rows of the filtered DataFrame
print(dev_dataframe_possible_list[:10])

Number of possible questions: 96
Number of possible questions: 197
Number of possible questions: 180
Number of possible questions: 108
Number of possible questions: 124
Number of possible questions: 217
Number of possible questions: 215
Number of possible questions: 241
Number of possible questions: 106
Number of possible questions: 231
Number of possible questions: 183
Number of possible questions: 170
Number of possible questions: 136
Number of possible questions: 104
Number of possible questions: 108
Number of possible questions: 116
Number of possible questions: 128
Number of possible questions: 197
Number of possible questions: 98
Number of possible questions: 113
Number of possible questions: 112
Number of possible questions: 96
Number of possible questions: 297
Number of possible questions: 176
Number of possible questions: 228
Number of possible questions: 215
Number of possible questions: 104
Number of possible questions: 153
Number of possible questions: 290
Number of possibl

Now we have a lot of samples to try our system

In [8]:
import json

row_index = 1

# Extract the relevant row
selected_row = dev_dataframe_possible.iloc[row_index]

# Convert the entire row to JSON and pretty print it
formatted_json = json.dumps(selected_row.to_dict(), sort_keys=False, indent=4)
print(formatted_json)

{
    "question": "When were the Normans in Normandy?",
    "id": "56ddde6b9a695914005b9629",
    "answers": [
        {
            "text": "10th and 11th centuries",
            "answer_start": 94
        },
        {
            "text": "in the 10th and 11th centuries",
            "answer_start": 87
        },
        {
            "text": "10th and 11th centuries",
            "answer_start": 94
        },
        {
            "text": "10th and 11th centuries",
            "answer_start": 94
        }
    ],
    "is_impossible": false
}


### Knowlege retrieval

**We have**: the questions (and the target answers)

**We need**: knowledge source & the retrieval system.


Installations of the required packages and libraries

In [53]:
!pip install -U sentence-transformers

In [54]:
!pip uninstall transformers
!pip install transformers==3.5

Found existing installation: transformers 4.36.2
Uninstalling transformers-4.36.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.36.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.36.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 25.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 21.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metada

In [58]:
!rm -rf ~/.cache/huggingface/transformers
!pip install -U sentence-transformers


  Using cached transformers-4.36.2-py3-none-any.whl (8.2 MB)


MS MARCO Passage Ranking is a large dataset to train models for information retrieval. It consists of about 500k real search queries from Bing search engine with the relevant text passage that answers the query.

* multi-qa-MiniLM-L6-cos-v1 maps sentences & paragraphs to a 384 dimensional dense vector space and was designed for semantic search. It has been trained on 215M (question, answer) pairs from diverse sources.

* cross-encoder/ms-marco-MiniLM-L-6-v2 model can be used for Information Retrieval: Given a query, encode the query with all possible passages (e.g. retrieved with ElasticSearch). Then sort the passages in a decreasing order


In [1]:
from sentence_transformers import SentenceTransformer, CrossEncoder

semb_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
xenc_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

### Answering a question

We are still missing the core of the QA system, the answering model.
We are going to re-use a pre-trained model.

In [ ]:
!pip install sentencepiece

In [ ]:
!pip -q install transformers sentencepiece accelerate

In [10]:
import torch
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
from transformers import AdamW
from torch.utils.data import DataLoader

# Load the pre-trained DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-cased')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Let's check quickly that it works:
zero shot learning: we give him a task he did not see during the training

In the zero-shot learning: evaluate the model on the SQuAD2.0 test set without providing any task-specific examples during testing. The model relies solely on its pre-trained knowledge and general language understanding to answer questions.

The tokenizer will automatically add any model specific separators (i.e. <CLS> and <SEP>) and tokens to the sentence.

TODO TOGLIERE

In [11]:
import torch

input_text = 'Translate the following sentence from Italian to English: "Amo la pizza"'

inputs = tokenizer(input_text, formatted_json, return_tensors="pt")
print(inputs)
# Make a prediction
with torch.no_grad():
    outputs = model(**inputs)


# Get predicted start and end indices
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

# Get the tokens corresponding to the answer span
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]

# Decode the predicted answer
predicted_answer = tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)
print("Predicted Answer:", predicted_answer)

{'input_ids': tensor([[  101, 13809,  8052,  1103,  1378,  5650,  1121,  2169,  1106,  1483,
           131,   107,  7277,  1186,  2495, 13473,   107,   102,   196,   107,
          2304,   107,   131,   107,  1332,  1127,  1103,  5177,  1116,  1107,
         13652,   136,   107,   117,   107, 25021,   107,   131,   107,  4376,
         13976,  2007,  1545,  1830,  1580,  1161,  1545,  1580,  1571,  1580,
         17175,  7629,  1571,  1830,  1580,  1545, 26752,   107,   117,   107,
          6615,   107,   131,   164,   196,   107,  3087,   107,   131,   107,
          5368,  1105,  5573,  3944,   107,   117,   107,  2590,   168,  1838,
           107,   131,  5706,   198,   117,   196,   107,  3087,   107,   131,
           107,  1107,  1103,  5368,  1105,  5573,  3944,   107,   117,   107,
          2590,   168,  1838,   107,   131,  5966,   198,   117,   196,   107,
          3087,   107,   131,   107,  5368,  1105,  5573,  3944,   107,   117,
           107,  2590,   168,  1838,  

# Index all the paragraphs and all the questions

In [12]:
import pandas as pd

# Replace 'path_al_tuo_file.json' with the actual file path
squad_data = dev_dataframe

# Extract questions and paragraphs and index them
questions = []
paragraphs = []

# Iterate over documents and paragraphs
for _, row in squad_data.iterrows():
    for paragraph in row['data']['paragraphs']:
        for qa in paragraph['qas']:
            question_dict = {
                'id': qa['id'],
                'title': row['data']['title'],
                'context': paragraph['context'],
                'question': qa['question'],
                'is_impossible': qa['is_impossible'],
                'answers': qa['answers']
            }
            questions.append(question_dict)

        # Append paragraphs to the separate list
        paragraph_dict = {
            'title': row['data']['title'],
            'context': paragraph['context']
        }
        paragraphs.append(paragraph_dict)

# Convert the lists to DataFrames
questions_df = pd.DataFrame(questions)
paragraphs_df = pd.DataFrame(paragraphs)

# Display the DataFrames
print("Questions:")
print(questions_df.head())

print("\nParagraphs:")
print(paragraphs_df.head())


Questions:
                         id    title  \
0  56ddde6b9a695914005b9628  Normans   
1  56ddde6b9a695914005b9629  Normans   
2  56ddde6b9a695914005b962a  Normans   
3  56ddde6b9a695914005b962b  Normans   
4  56ddde6b9a695914005b962c  Normans   

                                             context  \
0  The Normans (Norman: Nourmands; French: Norman...   
1  The Normans (Norman: Nourmands; French: Norman...   
2  The Normans (Norman: Nourmands; French: Norman...   
3  The Normans (Norman: Nourmands; French: Norman...   
4  The Normans (Norman: Nourmands; French: Norman...   

                                            question  is_impossible  \
0               In what country is Normandy located?          False   
1                 When were the Normans in Normandy?          False   
2      From which countries did the Norse originate?          False   
3                          Who was the Norse leader?          False   
4  What century did the Normans first gain their ...    

Now we can test our pipeline.
First select randomly a question from the data set

In [13]:
import random

random.seed(10)

idx = random.choice(range(len(questions)))

sample = questions[idx]
print(sample)
question = sample['question']
target_answer = sample['answers']

print(f'Question {idx}: {question}')
print(f'Answer {idx}: {target_answer}')

idx = random.choice(range(len(paragraphs)))

sample = paragraphs[idx]
print(sample)
context = sample['context']


print(f'Context {idx}: {context}')

{'id': '572fff45947a6a140053cf27', 'title': 'Rhine', 'context': "Most of the Rhine's current course was not under the ice during the last Ice Age; although, its source must still have been a glacier. A tundra, with Ice Age flora and fauna, stretched across middle Europe, from Asia to the Atlantic Ocean. Such was the case during the Last Glacial Maximum, ca. 22,000–14,000 yr BP, when ice-sheets covered Scandinavia, the Baltics, Scotland and the Alps, but left the space between as open tundra. The loess or wind-blown dust over that tundra, settled in and around the Rhine Valley, contributing to its current agricultural usefulness.", 'question': 'What stretched across middle Europe in the last ice age?', 'is_impossible': False, 'answers': [{'text': 'tundra', 'answer_start': 137}, {'text': 'tundra', 'answer_start': 137}, {'text': 'A tundra', 'answer_start': 135}]}
Question 9361: What stretched across middle Europe in the last ice age?
Answer 9361: [{'text': 'tundra', 'answer_start': 137}, 

# Embed paragraphs and questions

Now let's embed the retrieved passages (We can checkpoint the embeddings to avoid repeating the computation each time)

In [14]:
import os
import pickle

# Define hnswlib index path
embeddings_cache_path = './qa_embeddings_cache.pkl'

# Load cache if available
if os.path.exists(embeddings_cache_path):
    print('Loading embeddings cache')
    with open(embeddings_cache_path, 'rb') as f:
        corpus_embeddings = pickle.load(f)
# Else compute embeddings
else:
    print('Computing embeddings')
    corpus_embeddings = semb_model.encode(paragraphs, convert_to_tensor=True, show_progress_bar=True)
    # Save the index to a file for future loading
    print(f'Saving index to: \'{embeddings_cache_path}\'')
    with open(embeddings_cache_path, 'wb') as f:
        pickle.dump(corpus_embeddings, f)

Computing embeddings


Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Saving index to: './qa_embeddings_cache.pkl'


Finally let's index the embeddings (and let's save the index hoping it works)

In [16]:
!pip -q install hnswlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


* Hnswlib is a library for approximate nearest neighbor search used for efficient similarity search in high-dimensional spaces.
* We use cosine similarity to initialize the index. For each element in the corpus, hnswlib_index.add_items adds the corpus embeddings and their corresponding indices to the index.

In [17]:
import os
import hnswlib
import time
start = time.time()
# Create empthy index
index = hnswlib.Index(space='cosine', dim=corpus_embeddings.size(1))

# Define hnswlib index path
index_path = './qa_hnswlib_100.index'

# Load index if available
if os.path.exists(index_path):
    print('Loading index...')
    index.load_index(index_path)
# Else index data collection
else:
    # Initialise the index
    print('Start creating HNSWLIB index')
    index.init_index(max_elements=corpus_embeddings.size(0), ef_construction=100, M=64) # see https://github.com/nmslib/hnswlib/blob/master/ALGO_PARAMS.md for parameter description
    # Compute the HNSWLIB index (it may take a while)
    index.add_items(corpus_embeddings.cpu(), list(range(len(corpus_embeddings))))
    # Save the index to a file for future loading
    print(f'Saving index to: {index_path}')
    index.save_index(index_path)

end = time.time()
print(f"Exectution time: {int((end - start) / 60)}:{int((end - start) % 60)} min:sec")

Start creating HNSWLIB index
Saving index to: ./qa_hnswlib_100.index
Exectution time: 0:0 min:sec


In [18]:
import random

random.seed(80)

idx = random.choice(range(len(questions)))

sample = questions[idx]
question = sample['question']
target_answer = sample['answers']

print(f'Question {idx}: {question}?')

Question 4448: Inter-network routing was what kind of system??


We use the sentence embeddings model to encode the questions.

In [24]:
question_embedding = semb_model.encode(question, convert_to_tensor=True)

Retrieve relevant documents by using the index keeping top $k$ matches.

In [20]:
corpus_ids, distances = index.knn_query(question_embedding.cpu(), k=64)
scores = 1 - distances

print("Cosine similarity model search results")
print(f"Query: \"{question}\"")
print("---------------------------------------")
for idx, score in zip(corpus_ids[0][:5], scores[0][:5]):
    print(f"Score: {score:.4f}\nDocument: \"{paragraphs[idx]}\"\n\n")

Cosine similarity model search results
Query: "Inter-network routing was what kind of system?"
---------------------------------------
Score: 0.1563
Document: "{'title': 'Imperialism', 'context': 'Imperialism is a type of advocacy of empire. Its name originated from the Latin word "imperium", which means to rule over large territories. Imperialism is "a policy of extending a country\'s power and influence through colonization, use of military force, or other means". Imperialism has greatly shaped the contemporary world. It has also allowed for the rapid spread of technologies and ideas. The term imperialism has been applied to Western (and Japanese) political and economic dominance especially in Asia and Africa in the 19th and 20th centuries. Its precise meaning continues to be debated by scholars. Some writers, such as Edward Said, use the term more broadly to describe any system of domination and subordination organised with an imperial center and a periphery.'}"


Score: 0.1563
Docu

Re-rank retrieved documents using the cross-encoder.
As we can see now we obtain a better result (a document more related to the question).

In [21]:
import numpy as np

# Initialize texts list
texts = [[] for _ in range(len(paragraphs))]

# Populate texts list
for paragraph_idx, paragraph in enumerate(paragraphs):
    for idx, context_key in enumerate(['context']):  # Adjust keys as needed
        texts[paragraph_idx].append(paragraph[context_key])


# Create model inputs
model_inputs = [(question, ' '.join(paragraph_texts)) for paragraph_texts in texts]

# Predict scores using the cross-encoder model
cross_scores = xenc_model.predict(model_inputs)

# Print results
print("Cross-encoder model re-ranking results")
print(f"Query: \"{question}\"")
print("---------------------------------------")
for idx in np.argsort(-cross_scores)[:5]:
    print(f"Score: {cross_scores[idx]:.4f}\nDocument: \"{paragraphs[idx]['context']}\"\n\n")


Cross-encoder model re-ranking results
Query: "Inter-network routing was what kind of system?"
---------------------------------------
Score: 1.3056
Document: "AppleTalk was a proprietary suite of networking protocols developed by Apple Inc. in 1985 for Apple Macintosh computers. It was the primary protocol used by Apple devices through the 1980s and 90s. AppleTalk included features that allowed local area networks to be established ad hoc without the requirement for a centralized router or server. The AppleTalk system automatically assigned addresses, updated the distributed namespace, and configured any required inter-network routing. It was a plug-n-play system."


Score: -3.7926
Document: "There were two kinds of X.25 networks. Some such as DATAPAC and TRANSPAC were initially implemented with an X.25 external interface. Some older networks such as TELENET and TYMNET were modified to provide a X.25 host interface in addition to older host connection schemes. DATAPAC was developed by

Use best match to answer (and compare to reference answer)

In [22]:
passage_idx = np.argsort(-cross_scores)[0]

paragraphs = paragraphs[passage_idx]

input_text = f"Given the following paragrapha, answer the related question.\n\paragraphs:\n\n{paragraphs}\n\nQ: {question}?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
print(input_text, "\n")


Given the following paragrapha, answer the related question.
\paragraphs:

{'title': 'Packet_switching', 'context': 'AppleTalk was a proprietary suite of networking protocols developed by Apple Inc. in 1985 for Apple Macintosh computers. It was the primary protocol used by Apple devices through the 1980s and 90s. AppleTalk included features that allowed local area networks to be established ad hoc without the requirement for a centralized router or server. The AppleTalk system automatically assigned addresses, updated the distributed namespace, and configured any required inter-network routing. It was a plug-n-play system.'}

Q: Inter-network routing was what kind of system?? 



'\noutput_ids = model.generate(input_ids, max_new_tokens=32)\noutput_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)\nprint(output_text, "\n")\n\nprint(f"A (target): {target_answer}")\n'